# Python interface: practical guide

***Part 1: calling C++ from Python*** 

In this first part use the *spinfoam.py* module in order to call C++ source code implemented in the `./src` folder.
We do this by using the spinfoam shared library.

This allows us to take full advantage of the efficiency of the C++ code in order to deal with the most hard (computationally speaking) sections of the algorithm.

***Part 2: pure Python***   

In this second part we manipulate the csv files, computed and stored in the first part, with Python code.

## Step 0: computing the required fastwigxj hash tables

First, we must compute all the necessary hash tables of 3j, 6j and 9j Wigner symbols.
These tables are, in fact, a key ingredient for *step 1* and *step 2*.

For the calculation, we use the _wigxjpf_ and _fastwigxj_ libraries [Johansson et al., 2015], stored under `./ext/`.

In the following, with "type x" I mean "type x in linux shell terminal"

1. Change directory to `./ext/wigxjpf/`, type **make**.
2. Change directory to `./ext/fastwigxj/`, type **make**. 
   In in the followng, the latter is supposed to be the current directory.
   
3. Type "bin/hash_js --max-E-3j=50 /dev/null ./table_50.3j".
4. Type "bin/hash_js --max-E-6j=40 /dev/null ./table_40.6j".
5. Type "bin/gen_9j --flat-lim=16 | bin/combine_js | bin/unique_js ./comb_16.9j".
6. Type "bin/hash_js ./comb_16.9j ./hashed_16.9j".
7. Move the hash tables "hashed_16.9j", "table_40.6j" and "table_50.3j" generated in steps 3-6 in a directory of your choice.
   
   For example, create a folder `./data_folder/fastwigxj_tables/` and move all the hash tables in it.

   This is the default folder which I refer to in the following, but you can also choose another folder and provide a different 
   path.
   
**Important**: when using Jupyter notebook, if the hash table is not found in the provided path, the kernel dies with no message. 

When using python script from linux terminal, the C++ returns an error message which is correctly printed. 

Therefore, we must be sure that the fastwigxj hash tables above are actually stored before moving on.  

## Step 1: creating the **spinfoam** shared library

We first import the *spinfoam.py* module 

In [1]:
import spinfoam as sf

We can always clean all the previous binary and object files, as well as the shared library along with the corresponding folders, with the following cell

In [8]:
sf.spinfoam_clean()

cleaning wigxjpf...
rm -f *.test
rm -rf obj/
rm -rf gen/
rm -rf lib/
rm -rf bin/
rm -rf mod/
cleaning fastwigxj...
rm -rf gen/
rm -rf obj/
rm -rf lib/
rm -rf bin/
rm -rf mod/
rm -f test.3j.mark test.6j.mark test.9j.mark test_float128.6j.mark

rm -rf obj
rm -rf lib
rm -rf bin


We can compile and generate all the required files with the following cell, provided that *all dependencies are satisifed*.

For details, see the README file.

Notice that you can also compile in debug mode with **spinfoam_compile_debug()**

In [3]:
sf.spinfoam_compile()

compiling wigxjpf...
  AUTO   gen/wigxjpf_auto_config.h.TEST_LONG_DOUBLE
  AUTO   gen/wigxjpf_auto_config.h.TEST_FLOAT128
  AUTO   gen/wigxjpf_auto_config.h.TEST_THREAD
  AUTO   gen/wigxjpf_auto_config.h.TEST_UINT128
  AUTO   gen/wigxjpf_auto_config.h
   CC    obj/prime_factor.o
   CC    obj/calc.o
   CC    obj/trivial_zero.o
   CC    obj/c_wrap.o
   CC    obj/fortran_wrap.o
   CC    obj/wigxjpf_error.o
   CC    obj/calc_float128.o
   CC    obj/c_wrap_float128.o
   AR    lib/libwigxjpf_quadmath.a
   AR    lib/libwigxjpf.a
   CC    obj/wigxjpf.o
  LINK   bin/wigxjpf
compiling fastwigxj...
  AUTO   gen/fastwigxj_auto_config.h.TEST_LONG_DOUBLE
  AUTO   gen/fastwigxj_auto_config.h.TEST_FLOAT128
  AUTO   gen/fastwigxj_auto_config.h.TEST_THREAD
  AUTO   gen/fastwigxj_auto_config.h.TEST_THREAD_MUTEX
  AUTO   gen/fastwigxj_auto_config.h.TEST_LSFENCE
  AUTO   gen/fastwigxj_auto_config.h.TEST_SSE4_1
  AUTO   gen/fastwigxj_auto_config.h.TEST_SSE4_2
  AUTO   gen/fastwigxj_auto_config.h.TEST_AVX2
 

### Step 2: Hashing tables of 21j Wigner symbols 

We now compute the 21j Wigner symbols hash tables.
These  are a key ingredient for the Metropolis-Hastings algorithm, as they will be will be retrieved in *step 3*.

We use [parallel hashmap](https://github.com/greg7mdp/parallel-hashmap) in order to efficiently store and dump to disk the Wigner symbols. 

All the necessary _fastwigxj_ hash tables of 3j, 6j and 9j Wigner symbols must have been previously computed and stored in `./data_folder/fastwigxj_tables/` (or another folder of your choice, provided that the correct path is provided to the function below). See *step 0* for details

The C++ code searches for all files in the provided folder with extension ".3j", ".6j" or ".9j", and the largest tables found are set for loading.   

In [4]:
# this is the folder with the pre-computed hash tables of 3j, 6j and 9j Wigner symbols.
# Jupyter kernel dies if tables are not found (C function returns "EXIT FAILURE")
fastwigxj_tables_path = "./data_folder/fastwigxj_tables"

# this is the folder in which we wanto to store the computed 21j symbols hash tables
hash_tables_path = "./data_folder/21j_hash_tables"

In [5]:
# 'spin' is the value of all boundary spins of the spinfoam

for spin in [0.5, 1, 1.5, 2]:
    sf.Hashing_21j_symbols(hash_tables_path, fastwigxj_tables_path, spin)

Hashing all 21j symbols with j <= 0.5...
Maximum amount is roughly 5184 symbols, starting... 
Completed! The hash table has been stored
Hashing all 21j symbols with j <= 1...
Maximum amount is roughly 103680 symbols, starting... 
Completed! The hash table has been stored
Hashing all 21j symbols with j <= 1.5...
Maximum amount is roughly 960000 symbols, starting... 
Completed! The hash table has been stored
Hashing all 21j symbols with j <= 2...
Maximum amount is roughly 5670000 symbols, starting... 
Completed! The hash table has been stored


### Step 3: Metropolis-Hastings 

We can now compute and assemble the Markov chains by running the MH algorithm. The computed draws are stored in the given folder.

**We run in parallel an independent Markov chain for each provided thread**.

*It is advisable for the performance to use a number of Markov chains equal to or less than the physical number of cores present on the system.*

Each thread retrieves from the 21j hash table (computed in step 2) the required symbol.

In [6]:
# folder in which we want to store the data
data_folder = "./data_folder/data"

# this is the folder with the pre-computed hash tables of 21j Wigner symbols (see Step 2).
# Jupyter kernel dies if table is not found (C function returns "EXIT FAILURE")
# hash_tables_path = "./data_folder/21j_hash_tables"

# MH parameters
spin = 0.5
length = 1000000
sigma = 0.40
burnin = 12
verbosity = 0

# set optimal number of threads
import multiprocessing

number_of_threads = int(multiprocessing.cpu_count())
print(f"optimal number of threads: {number_of_threads}")

optimal number of threads: 12


In [7]:
sf.Metropolis_Hastings_parallel_run(
    data_folder,
    hash_tables_path,
    spin,
    length,
    sigma,
    burnin,
    verbosity,
    number_of_threads,
)

Starting 12 independent Markov chains...
Completed! All draws have been stored


### Step 4: Manipulate the CSV file with Python

We can now manipulate the csv files, computed in *step 3*, in order to extract the informations we are interested in.

See `notebooks` folder